In [1]:
# STEP 1: Install Java and Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz

# STEP 2: Set environment variables manually
import os
import sys

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# STEP 3: Append PySpark to Python path manually
sys.path.append("/content/spark-3.3.2-bin-hadoop3/python")
sys.path.append("/content/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip")

# STEP 4: Install PySpark
!pip install -q pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Indian Car Accident Analysis") \
    .getOrCreate()


In [3]:
from google.colab import files
uploaded = files.upload()


Saving combined_accident_data.csv to combined_accident_data.csv


In [4]:
df = spark.read.csv("combined_accident_data.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)


root
 |-- Num_Acc: string (nullable = true)
 |-- inverse_data: timestamp (nullable = true)
 |-- week_day: string (nullable = true)
 |-- state: string (nullable = true)
 |-- severity: string (nullable = true)
 |-- weather: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hrmn: integer (nullable = true)
 |-- lum: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- driver_sex: string (nullable = true)
 |-- driver_age: integer (nullable = true)
 |-- car_age: integer (nullable = true)
 |-- casualty_severity: string (nullable = true)
 |-- casualty_age: integer (nullable = true)
 |-- casualty_sex: string (nullable = true)
 |-- social_class: string (nullable = true)
 |-- casualty_type: string (nullable = true)
 |-- pedestrian_or_passenger: string (nullable = true)
 |-- severity_description: string (nullable = true)
 |-- weather_description: string (nullable = true

In [5]:
# 2. Count of total rows and missing values per column
print("Total rows:", df.count())
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Total rows: 1000
+-------+------------+--------+-----+--------+-------+--------+----+---+------------+-------------+-----------+----------+----------+-------+-----------------+------------+------------+------------+-------------+-----------------------+--------------------+-------------------+--------------------+------------------------+-----------------------+----------------------+-----------------------------+-------------------------+
|Num_Acc|inverse_data|week_day|state|severity|weather|location|hrmn|lum|vehicle_type|vehicle_model|engine_size|driver_sex|driver_age|car_age|casualty_severity|casualty_age|casualty_sex|social_class|casualty_type|pedestrian_or_passenger|severity_description|weather_description|location_description|vehicle_type_description|engine_size_description|driver_sex_description|casualty_severity_description|casualty_type_description|
+-------+------------+--------+-----+--------+-------+--------+----+---+------------+-------------+-----------+----------+-------

In [7]:
# Accidents by state or region
accidents_by_state = df.groupBy("state").count().orderBy("count", ascending=False)
accidents_by_state.show(10)

+-----+-----+
|state|count|
+-----+-----+
|   TR|   48|
|   DL|   44|
|   GJ|   44|
|   UP|   44|
|   HP|   43|
|   AS|   40|
|   ML|   39|
|   CG|   38|
|   AP|   38|
|   WB|   37|
+-----+-----+
only showing top 10 rows



In [9]:
df.columns

['Num_Acc',
 'inverse_data',
 'week_day',
 'state',
 'severity',
 'weather',
 'location',
 'hrmn',
 'lum',
 'vehicle_type',
 'vehicle_model',
 'engine_size',
 'driver_sex',
 'driver_age',
 'car_age',
 'casualty_severity',
 'casualty_age',
 'casualty_sex',
 'social_class',
 'casualty_type',
 'pedestrian_or_passenger',
 'severity_description',
 'weather_description',
 'location_description',
 'vehicle_type_description',
 'engine_size_description',
 'driver_sex_description',
 'casualty_severity_description',
 'casualty_type_description']

In [11]:
# Accidents over Time

from pyspark.sql.functions import year, month, to_date

# Convert date string to proper date type (won’t drop nulls)
df_with_date = df.withColumn("date", to_date(col("inverse_data"), "yyyy-MM-dd"))

# Group by Year and Month
df_with_date.groupBy(
    year("date").alias("year"),
    month("date").alias("month")
).count().orderBy("year", "month").show()



+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2022|    1|   31|
|2022|    2|   33|
|2022|    3|   50|
|2022|    4|   41|
|2022|    5|   45|
|2022|    6|   42|
|2022|    7|   43|
|2022|    8|   38|
|2022|    9|   38|
|2022|   10|   49|
|2022|   11|   41|
|2022|   12|   38|
|2023|    1|   38|
|2023|    2|   38|
|2023|    3|   48|
|2023|    4|   39|
|2023|    5|   43|
|2023|    6|   28|
|2023|    7|   40|
|2023|    8|   43|
+----+-----+-----+
only showing top 20 rows



In [12]:
# Most common vehicle types involved

df.groupBy("vehicle_type").count().orderBy("count", ascending=False).show(10)

+------------+-----+
|vehicle_type|count|
+------------+-----+
|  Motorcycle|  263|
|       Truck|  263|
|         Car|  240|
|         Bus|  234|
+------------+-----+



In [13]:
# Casuality Severity

df.groupBy("casualty_severity").count().orderBy("count", ascending=False).show()

+-----------------+-----+
|casualty_severity|count|
+-----------------+-----+
|          Serious|  349|
|           Slight|  326|
|            Fatal|  325|
+-----------------+-----+



In [15]:
# Weather Conditions during Accidents
df.select("weather_description").groupBy("weather_description") \
  .count().orderBy("count", ascending=False).show(truncate=False, n=30)

+----------------------------------------------------------------------+-----+
|weather_description                                                   |count|
+----------------------------------------------------------------------+-----+
|Weather conditions during the accident (Clear, Rain, Fog, Snow, Storm)|1000 |
+----------------------------------------------------------------------+-----+



In [17]:
# Accidents per weather type
df.groupBy("weather").count().orderBy("count", ascending=False).show()



+-------+-----+
|weather|count|
+-------+-----+
|    Fog|  223|
|   Rain|  218|
|  Clear|  194|
|   Snow|  183|
|  Storm|  182|
+-------+-----+



In [19]:
# Severity and Weather
df.groupBy("weather", "severity").count().orderBy("weather", "count", ascending=False).show()


+-------+--------+-----+
|weather|severity|count|
+-------+--------+-----+
|  Storm|Moderate|   77|
|  Storm|  Severe|   53|
|  Storm|   Minor|   52|
|   Snow|Moderate|   63|
|   Snow|  Severe|   62|
|   Snow|   Minor|   58|
|   Rain|Moderate|   77|
|   Rain|   Minor|   77|
|   Rain|  Severe|   64|
|    Fog|  Severe|   84|
|    Fog|   Minor|   72|
|    Fog|Moderate|   67|
|  Clear|   Minor|   76|
|  Clear|  Severe|   66|
|  Clear|Moderate|   52|
+-------+--------+-----+



In [21]:
# Average Age of Driver by Severity
from pyspark.sql.functions import avg

df.groupBy("severity").agg(avg("driver_age").alias("avg_driver_age")).orderBy("severity").show()


+--------+------------------+
|severity|    avg_driver_age|
+--------+------------------+
|   Minor|44.265671641791045|
|Moderate| 43.87202380952381|
|  Severe| 44.57142857142857|
+--------+------------------+



## Key Insights:
- Most accidents occurred during foggy weather.
- States like Tripura, Delhi, Gujarat had the highest accident counts.
- Motorcycle and Trucks were the most involved in Accidents.
- Accidents were high during March and October months (both years), with March showing the highest accident counts overall.
- Accident counts in the first 8 months of 2023 are slightly lower or comparable to the same months in 2022.
- The count of fatal casualties (325) are nearly equal to slight injuries, which highlights a significant number of deadly accidents in the dataset.